In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
import tensorflow as tf
import collections as cl
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
print("SETUP COMPLETE!")

In [ ]:
data = pd.read_csv('../input/all-space-missions-from-1957/Space_Corrected.csv')
print('DATA IMPORTING SUCCESFULLY!')

In [ ]:
print('PRINTING THE DATA')
data

In [ ]:
print('PREPROCESSING, REMOVING DUPLICATED OR REDUNDANT DATA!')
data.drop([data.columns[0], data.columns[1], 'Location', 'Detail'], axis=1, inplace=True)

In [ ]:
print('PRINTING THE DATA AFTER PREPROCESSING')
data

In [ ]:
data.columns

In [ ]:
print('REMOVING THE SPACE FROM ROCKET COLUMN')
data.columns = ['Company Name', 'Datum', 'Status Rocket', 'Rocket', 'Status Mission']

In [ ]:
print('CHECKING NULL DATA')
data.isnull().sum()

In [ ]:
print ('CHECKING ROCKET VALUES')
data['Rocket'].unique()

In [ ]:
print('CONVERTING AND FORMATTING THE DATA')
data['Rocket'].astype(str).apply(lambda data_format: data_format.replace(',', '')).astype(np.float32)

In [ ]:
print('ASSIGNING CONVERTED VALUES')
data['Rocket'] = data['Rocket'].astype(str).apply(lambda data_format: data_format.replace(',', '')).astype(np.float32)

In [ ]:
print('FILLING MISSING VALUES')
data['Rocket'] = data['Rocket'].fillna(data['Rocket'].mean())

In [ ]:
print('CHEKING MISSING VALUES')
data.isnull().sum()

In [ ]:
print('EXTRACTING THE YEAR')
def get_year(date):
    year = re.search(r'[^,]*$', date).group(0)
    year = re.search(r'^\s[^\s]*', year).group(0)
    return np.int16(year)

In [ ]:
print('CHECKING THE OUTPUT FOR A NUMBER WITH SPACE')
np.int16(' 2020')

In [ ]:
print('EXTRACTING THE MONTH')
def get_month(date):
    month = re.search(r'^[^0-9]*', date).group(0)
    month = re.search(r'\s.*$', month).group(0)
    return month.strip()

In [ ]:
print('APPLYING get_year AND get_month FUNCTIONS')
data['Year'] = data['Datum'].apply(get_year)
data['Month'] = data['Datum'].apply(get_month)
data.drop('Datum', axis=1, inplace=True)

In [ ]:
print('CHECKING RESULTS')
data

In [ ]:
print('CHECKING STATUS MISSION')
data['Status Mission'].unique()

In [ ]:
print('CONVERTING PRELAUNCH FAILURE & PARTIAL FAILURE TO FAILURE')
data['Status Mission'] = data['Status Mission'].apply(lambda mission_status: mission_status if mission_status == 'Success' else 'Failure')

In [ ]:
print('CHECKING STATUS MISSION')
data['Status Mission'].unique()

In [ ]:
print('COVERTING STATUS MISSION TO BOOL')
encoder = LabelEncoder()
data['Status Mission'] = encoder.fit_transform(data['Status Mission'])

In [ ]:
print('CHECKING STATUS ROCKET')
data['Status Rocket'].unique()

In [ ]:
print('ORDERING STATUS ROCKET')
rocket_ordering = ['StatusRetired', 'StatusActive']

In [ ]:
print('ORDERING MONTH')
month_ordering = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']


In [ ]:
print('ORDERING FUNCTION')
def ordering(data, column, order):
    return data[column].apply(lambda new_order: order.index(new_order))

In [ ]:
print('APPLYING ORDERING')
data['Month'] = ordering(data, 'Month', month_ordering)
data['Status Rocket'] = ordering(data, 'Status Rocket', rocket_ordering)

In [ ]:
data

In [ ]:
print('TOTAL:', data['Status Mission'].count())

In [ ]:
print('0 = FAILURE, 1 = SUCCESS')
data['Status Mission'].value_counts().plot(kind='barh')

In [ ]:
print('CREATING A MATRIX BASED TO BE USED THROUGH COLUMNS')
def onehot(data, column):
    dummies = pd.get_dummies(data[column])
    data = pd.concat([data, dummies], axis=1)
    data.drop(column, axis=1, inplace=True)
    return data

In [ ]:
print('USING THE FUNCTION TO PRINT A NEW TABLE THROUGH THE MATRIX')
data = onehot(data, 'Company Name')
data

In [ ]:
print('SCALLING')
y = data['Status Mission']
x = data.drop('Status Mission', axis=1)

In [ ]:
print('SET THE VALUE BETWEEN 0 ~ 1')
scaler = MinMaxScaler()
x = pd.DataFrame(scaler.fit_transform(x), columns=x.columns)

In [ ]:
print('PRINTING THE TRAINING MODEL')
x

In [ ]:
print('IDENTIFYING THE METRICS')
y.sum() / len(y)
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7)

In [ ]:
print('CALCULATING DECIMALS TO APPROACH THEM TO THEIR CLOSEST INTEGER')
inputs = tf.keras.Input(shape=(60,))
x = tf.keras.layers.Dense(16, activation='relu')(inputs)
x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
print('COMPILING AND TRAINING THE MODEL')
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.AUC(name='auc')]
)


batch_size=32
epochs=50

history = model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    batch_size=batch_size,
    epochs=epochs
)

In [ ]:
print('PLOTTING THE DATA')
plt.figure(figsize=(14, 10))

epochs_range = range(1, epochs + 1)
train_loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(epochs_range, train_loss, label="Training Loss")
plt.plot(epochs_range, val_loss, label="Validation Loss")

plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend('upper right')

plt.show()

In [ ]:
print('LOST DATA')
np.argmin(val_loss)

In [ ]:
print('CHECKING ACCURACY')
model.evaluate(x_test, y_test)